<a href="https://colab.research.google.com/github/yash-1708/Stock-Analyst-Rating/blob/master/scrapingmarketsmojoBIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [2]:
driver = webdriver.Chrome("chromedriver.exe")
broker=[] #List to store name of the broker
company_name=[] #List to store company names 
current_price=[] #List to store rating of the product
predict_date=[]
recomended_buying=[]
target_price=[]
driver.get("https://www.marketsmojo.com/recommendation/")


In [3]:
content = driver.page_source
soup = BeautifulSoup(content,'html.parser')
Alldata=soup.select_one("div.righttexcontent2 div.right-table01 div.scr-result2")
# for row in Alldata.findAll('div',class_='middle-colum02'):
for row in Alldata.findAll('div',attrs={'class':'middle-colum02'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)
    #predict_date.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott"))
     
for row in Alldata.findAll('div',attrs={'class':'middle-columactive'}):
    broker.append(row.select("div.colum2-midd01 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text())
    company_name.append(row.select("div.colum2-midd02 div.row div.margin-text ul.reco-price li div.text-top0 a.broker-link")[0].get_text().replace('\t','').replace('\n',''))
    current_price.append(row.select("div.colum2-midd03 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    recomended_buying.append(row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.text-top0")[0].get_text())
    target_price.append(row.select("div.colum2-midd05 div.row div.margin-text ul.reco-price li div.text-top0")[0].get_text())
    temp_data=row.select("div.colum2-midd04 div.row div.margin-text ul.reco-price div.colum-text-color-bott")
    temp_data=str(temp_data).split('on ')[1]
    temp_data=temp_data.split('<br/')[0]
    predict_date.append(temp_data)

In [4]:
temp_data

'08 Aug 19'

In [5]:
df1 = pd.DataFrame({'broker':broker})  
df2 = pd.DataFrame({'company_name':company_name})
df3 = pd.DataFrame({'current_price':current_price})
df4 = pd.DataFrame({'recomended_buing':recomended_buying})
df5=pd.DataFrame({'target_price':target_price})
df6=pd.DataFrame({'predict_date':predict_date})

In [6]:
rb=df4['recomended_buing'].str.split('Rs.').str[1] 
cp=df3['current_price'].str.split('Rs.').str[1]
tp=df5['target_price'].str.split('Rs.').str[1]
predict_date=df6['predict_date'].str.split('10').str[0]

In [7]:
dfall=pd.DataFrame({'broker':broker, 'company_name':company_name, 'current_price':cp, 'recomended_buying':rb, 'predict_date':predict_date, 'target_price':tp})
dfallclean = dfall[dfall['predict_date']!='']
dfallclean['predict_date'] =  pd.to_datetime(dfallclean['predict_date'], format='%d %b %y')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
dfallclean

broker          company_name current_price  \
0        Prabhudas Lilladher         Indusind Bank        1498.5   
1              Motilal Oswal                Voltas        680.85   
2        Prabhudas Lilladher          L&T Infotech        1649.1   
3     LKP Securities Limited        Sun TV Network        458.55   
4            ICICIdirect.com          L&T Infotech        1649.1   
...                      ...                   ...           ...   
3753     Prabhudas Lilladher                 Cipla         462.5   
3754     Prabhudas Lilladher          Petronet LNG         274.4   
3755     Prabhudas Lilladher            Tata Steel        424.85   
3756     Karvy Stock Broking  Tata Global Beverage        323.55   
3757     Karvy Stock Broking            Bata india       1710.45   

     recomended_buying predict_date target_price  
0              1472.12   2019-12-15      1640.00  
1               681.19   2019-12-13       785.00  
2              1630.00   2019-12-12      1825.00  
3               460.79   2019-12-12       596.00  
4              1648.74   2019-12-12      1880.00  
...                ...          ...          ...  
3753            519.00   2019-08-08       439.00  
3754            235.35   2019-08-08       306.00  
3755            381.80   2019-08-08       350.00  
3756            270.10   2019-08-08       263.00  
3757           1400.10   2019-08-08      1646.00  

[3710 rows x 6 columns]

In [19]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [20]:
maxdate = pd.read_sql('SELECT MAX(predict_date) FROM marketsmojorecos', con=database_connection)

In [21]:
maxdate

MAX(predict_date)
0        2019-12-15

In [12]:
a=maxdate.loc[0,'MAX(predict_date)']

In [14]:
date_object1=maxdate['MAX(predict_date)']
#date_object1.str
mynew=str(a)

In [15]:
date_object1 = datetime.strptime(mynew, '%Y-%m-%d').date()

In [16]:
mask = (dfallclean['predict_date'] > date_object1) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [17]:
dfentry = dfallclean.loc[mask]
dfentry

broker       company_name current_price  \
0           Prabhudas Lilladher      Indusind Bank        1498.5   
1                 Motilal Oswal             Voltas        680.85   
2           Prabhudas Lilladher       L&T Infotech        1649.1   
3        LKP Securities Limited     Sun TV Network        458.55   
4               ICICIdirect.com       L&T Infotech        1649.1   
...                         ...                ...           ...   
3526        Prabhudas Lilladher               BPCL           493   
3527                Axis Direct      Bajaj Finance       4056.35   
3528  Geojit Financial Services           Hindalco         207.6   
3529  Geojit Financial Services               HDFC          2368   
3530  Geojit Financial Services  Grasim Industries         768.2   

     recomended_buying predict_date target_price  
0              1472.12   2019-12-15      1640.00  
1               681.19   2019-12-13       785.00  
2              1630.00   2019-12-12      1825.00  
3               460.79   2019-12-12       596.00  
4              1648.74   2019-12-12      1880.00  
...                ...          ...          ...  
3526            378.64   2019-09-05       466.00  
3527           3243.58   2019-09-04      3591.00  
3528            179.21   2019-09-04       195.00  
3529           2096.68   2019-09-04      2284.00  
3530            694.82   2019-09-04       803.00  

[2915 rows x 6 columns]

In [18]:

dfentry.to_sql(con=database_connection, name='marketsmojorecos', if_exists='append')